In [ ]:
!pip install gradio_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from gradio_client import Client
client = Client("https://lykeven-cogvlm.hf.space/")

Loaded as API: https://lykeven-cogvlm.hf.space/ ✔


In [ ]:
import json
import pandas as pd
import re
import ast
import numpy as np

Make the JSON file

In [ ]:
def file_content(result):
  with open(result,'r') as file:
    data=json.load(file)
  return json.dumps(data, indent=4)

In [ ]:
result = client.predict(
				"Howdy!",	# str in 'parameter_7' Button component
				fn_index=4
)
print(file_content(result))

[
    [
        "",
        "Hi, What do you want to know about this image?"
    ]
]


Find the No of Likes


In [ ]:
def format_tweet(tweet, date, username, company):
  return f"""
      Tweet: {tweet}
      Date: {date}
      Username: {username}
      Company: {company}

  """

In [ ]:
def get_formatted(tweets):
  ans=[]
  for i in tweets:
    row=data.iloc[i:i+1,:].values[0]
    ans.append(format_tweet(row[3],row[1],row[4],row[6]))
  return ans

In [ ]:
init_prompt = "You are a highly knowledgable person who is very active on twitter, now you have to predict the number of likes given a tweet and the username."

In [ ]:
finish_prompt = "Make sure to give an exact number and only the number as an integer without any explanation or reasoning behind it."

In [ ]:
def find_image_url(url):
  if url[1]=='P':
    pattern = r"fullUrl='(.*?)'"
    match = re.search(pattern, url)
    return match.group(1)
  pattern = r"thumbnailUrl='(.*?)'"
  match = re.search(pattern, url)
  return match.group(1)

In [ ]:
def predict_likes(row):
  global result
  img_url=find_image_url(row[5])
  prompt=init_prompt+format_tweet(row[3],row[1],row[4],row[6])+finish_prompt
  result = client.predict(
				prompt,	# str in 'Input Text' Textbox component
				0.8,	# int | float (numeric value between 0 and 1)
				0.4,	# int | float (numeric value between 0 and 1)
        img_url,	# str (filepath or URL to image)
        result,	# str (filepath to JSON file)
				"",	# str in 'parameter_21' Textbox component
				False,	# bool in 'Grounding' Checkbox component
				fn_index=1
  )
  print(prompt)
  result=result[1]
  data=ast.literal_eval(file_content(result))[0][1]
  if data[0] == 'T':
    return '-1'
  return data

In [ ]:
data=pd.read_excel('/content/drive/MyDrive/behaviour_simulation_train.xlsx')

In [ ]:
def predict(start,end):
  while start<end:
    row=data.iloc[start:start+1,:].values
    row=row[0]
    ans=predict_likes(row)
    if ans != '-1':
      print(f"Predicted: {ans} Expected: {row[2]}")
      start+=1

In [ ]:
predict(0,2)

      You are a highly knowledgable person who is very active on twitter, now you have to predict the number of likes given a tweet and the username.
      Tweet: Spend your weekend morning with a Ham, Egg, and Cheese Wrap paired with a sweet Iced French Vanilla! ☀️ Order yours now via dine-in, takeout, and delivery. #TimHortonsPH <hyperlink>
      Date: 2020-12-12 00:47:00
      Username: TimHortonsPH
      Company: tim hortons

  Make sure to give an exact number and only the number without any explanation
Predicted: 1000 Expected: 1
      You are a highly knowledgable person who is very active on twitter, now you have to predict the number of likes given a tweet and the username.
      Tweet: Watch rapper <mention> freestyle for over an HOUR <hyperlink> <hyperlink>
      Date: 2018-06-30 10:04:20
      Username: IndyMusic
      Company: independent

  Make sure to give an exact number and only the number without any explanation
Predicted: 100K Expected: 2750


Vector DB

In [ ]:
!pip install pinecone-client langchain sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d2ec29fcbdb7c15b90bc52dae4f87f6b009dbb93318e91e88e9ba6b40087f40c
  Stored in directory: /root/.cache/pip/wheels/62/f

Model for Embedding


In [ ]:
import pinecone
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import torch

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32},
    multi_process=True  #False?
)

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
docs=["Hello"]
word_embedding_dimension = len(embed_model.embed_documents(docs)[0])

In [ ]:
def create_embeddings(start,end):
  docs=data.iloc[start:end,:].values
  vectors=[]
  for row in docs:
    vectors.append(format_tweet(row[3],row[1],row[4],row[6]))
  return embed_model.embed_documents(vectors)

Create the Index


In [ ]:
import time

def create_index(index_name,pinecone):

  if index_name not in pinecone.list_indexes():
      pinecone.create_index(
          name=index_name,
          dimension=word_embedding_dimension,
          metric='cosine'
      )
      # wait a moment for the index to be fully initialized
      while not pinecone.describe_index(index_name).status['ready']:
          time.sleep(1)

  index = pinecone.Index(index_name)
  print(index.describe_index_stats())

Inserting Vectors

In [ ]:
def insert_vectors(start,embeddings,index):
  vectors=[]
  for i in embeddings:
    vectors.append({"id":str(start),"values":i})
    start+=1
  index.upsert(vectors)

Query

In [ ]:
def query(data_ind,index):
  row=data.iloc[data_ind:data_ind+1,:].values[0]
  text=format_tweet(row[3],row[1],row[4],row[6])
  vector=embed_model.embed_documents([text])
  matches=index.query(
    vector=vector,
    top_k=3,
    include_values=False
  )['matches']
  ids=[]
  for i in matches:
    ids.append([int(i['id']),i['score']])
  return ids

Delete Index

In [ ]:
def delete(index_name,pinecone):
  pinecone.delete_index(index_name)

Insert Bunch of vectors


In [ ]:
def putIntoDB(start,end,index):
  embeddings=create_embeddings(start,end)
  insert_vectors(start,embeddings,index)

Create the index

Insert into The Vector DB

In [ ]:
api_keys=['c2dd68f5-5e0d-46a8-8c1e-36052d498b25','bbcdb566-41cd-4bd2-8f70-d5b6e33b9463','924c40de-3709-4fc4-8c71-c0d748a2c561']
env='gcp-starter'
index_name='vector-search'

for i in range(0,3):
  api_key=api_keys[i]
  pinecone.init(api_key=api_key,environment=env)
  create_index(index_name,pinecone)
  index=pinecone.Index(index_name)

  for j in range(0,100000,1000):
    start_idx=i*100000+j
    putIntoDB(start_idx,start_idx+1000,index)
    print(start_idx+1000)

Run the queries

In [ ]:
ans=query(247329,index)
print(ans)

[[247329, 1.00025952], [215775, 0.796966374], [297584, 0.793590844]]


In [ ]:
index=pinecone.Index(index_name)
print(index.describe_index_stats())

{'dimension': 384,
 'index_fullness': 1.0,
 'namespaces': {'': {'vector_count': 100000}},
 'total_vector_count': 100000}
